## Metrics: calculating_03: Controlling tf.metrics()

<font color=red>TODO: add links and information on the series here </font>

#### Series information
- previous: tf.metrics() introduction [video]() [notebook]()
- next: controlling the metrics [video]() [notebook]()

#### Related series
- metrics [video]() [notebook]()|

#### Resources:
 - [official tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf/metrics)

In [1]:
import tensorflow as tf
import numpy as np
import os

# Helper to make the output consistent
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
# supress most messages (display only error messages)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Check TensorFlow version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU found')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
# reset and seed graph
reset_graph()

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


In [2]:
x = tf.placeholder(tf.int32, [10])
y = tf.placeholder(tf.int32, [10])

# fake data
# batch acc = 20%
y_pred_cls = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
y_true_cls = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Metrics
We'll again use accuracy (see [here]() for an introduction to `tf.metrics.accuracy()`)

In [3]:
with tf.variable_scope("accuracy_metrics") as scope:
    acc, acc_op = tf.metrics.accuracy(labels=y, predictions=x)
    m_vars = tf.get_variable(collection=tf.GraphKeys.LOCAL_VARIABLES)
    acc_reset_op = tf.variables_initializer(m_vars)

In [4]:
# initializers, global and local
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

In [5]:
# initialize the session and global and local vars
# note: we will want to close the session at the end `sess.close()`
sess = tf.Session()
init_global.run(session=sess)
init_local.run(session=sess)

In [12]:
v = sess.run(acc, feed_dict={x: y_pred_cls, y: y_true_cls})
print("Current accuracy: {:.2f}%".format(v*100))
             
v = sess.run(acc_op, feed_dict={x: y_pred_cls, y: y_true_cls})
             
# batch acc = 40%
y_pred_cls = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1])
y_true_cls = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
v = sess.run(acc_op, feed_dict={x: y_pred_cls, y: y_true_cls})
             
# batch acc = 60%
y_pred_cls = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
y_true_cls = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
v = sess.run(acc_op, feed_dict={x: y_pred_cls, y: y_true_cls})

# total should be 40% ((2+4+6)/(10+10+10))
v = sess.run(acc, feed_dict={x: y_pred_cls, y: y_true_cls})
print("Current accuracy: {:.2f}%".format(v*100))

Current accuracy: 45.00%
Current accuracy: 48.00%
Current accuracy: 48.57%


### What if we want to reset the metrics (say for another batch/epoch?)

In [13]:
v = sess.run(acc, feed_dict={x: y_pred_cls, y: y_true_cls})
print("accuracy pre reset: {:.2f}%".format(v*100))

sess.run(acc_reset_op)

v = sess.run(acc, feed_dict={x: y_pred_cls, y: y_true_cls})
print("accuracy post reset: {:.2f}%".format(v*100))

accuracy pre reset: 48.57%
accuracy post reset: 0.00%


### How does `acc_reset_op` work?
we're using;
```
tf.variable_scope()
tf.get_variable()
collection=tf.GraphKeys.LOCAL_VARIABLES
tf.variables_initializer()
```
first, we're using the functionality within the given scope.  Then we access the target variable(s) with `tf.get_variable()`, and specify that we only want the local variables with the parameter `collection=tf.GraphKeys.LOCAL_VARIABLES`, then we (re)initialize them.

All together, it looks like this;

```python
m_vars = tf.get_variable(collection=tf.GraphKeys.LOCAL_VARIABLES)
acc_reset_op = tf.variables_initializer(m_vars)
```

We can then use `sess.run(acc_reset_op)` to reset the variable.